# Abigeato, Amenazas & Delitos Sexuales

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
PATH_FILE = '/content/drive/MyDrive/Police-Data/Bronce'
OUTPUT_PATH_FILE = '/content/drive/MyDrive/Police-Data/Silver'
CODIGO_DANE_PATH = '/content/drive/MyDrive/Police-Data/dane_codes.xlsx'

## Libraries

In [3]:
import pandas as pd
import numpy as np
import os
import glob

## Read Files

In [4]:
def find_header_row(file_path):
    """
    Finds the header line with a column name
    """
    column_name = ['DEPARTAMENTO','DEPARTAMENTO ','Departamento','departamento']
    header_row = None

    df = pd.read_excel(file_path, header=None)

    for i, row in df.iterrows():
        if column_name[0] in row.values or column_name[1] in row.values or column_name[2] in row.values or column_name[3] in row.values:
            header_row = i
            break
        if i>20:
            break

    if header_row is None:
        raise ValueError(f"No se encontró la fila de encabezado en el archivo {file_path}")

    return header_row

def read_excel_with_variable_header(file_path):
    header_row = find_header_row(file_path)

    df = pd.read_excel(file_path, header=header_row)

    total_row_index = df[df.iloc[:, 0].str.contains("TOTAL", case=False, na=False)].index.min()

    if total_row_index is not None:
        df = df.iloc[:total_row_index]

    return df

In [5]:
folders = ["Abigeato","Amenazas","Delitos_Sexuales"]

dataframes_by_folder = {}

def folder_read(folders):
  for folder in folders:
      file_paths = glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xlsx')) + glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xls'))
      folder_dataframes = []
      for file_path in file_paths:
          try:
              df = read_excel_with_variable_header(file_path)
              folder_dataframes.append(df)
          except ValueError as e:
              print(e)
      dataframes_by_folder[folder] = folder_dataframes

In [6]:
folder_read(folders)

## Abigeato

In [7]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [8]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Abigeato']]
abigeato_df = pd.concat(standardized_dataframes, ignore_index=True)

In [9]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_no_valid_values = ["-"]
abigeato_df["AGRUPA_EDAD_PERSONA"] = abigeato_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
abigeato_df["GENERO"] = abigeato_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
abigeato_df["ARMAS_MEDIOS"] = abigeato_df["ARMAS_MEDIOS"].apply(lambda x: x if x not in armas_medios_no_valid_values else "NO REPORTADO")

In [10]:
dane_codes = pd.read_excel(CODIGO_DANE_PATH,sheet_name='Municipios_dane',dtype={'CODIGO_DANE':str})
def clean_data(data):
  temp = data.copy()
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].astype(str)
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].apply(lambda x: '0' + x if len(x) == 9 else x)
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].str[:5]
  filtered_df = temp[temp['CODIGO_DANE'].apply(lambda x: x[-1].isdigit() if x else False)]
  # Adjust upper case
  filtered_df = filtered_df.applymap(lambda x: x.upper() if isinstance(x, str) else x)
  # Merge DANE CODE
  merged_df = filtered_df.merge(dane_codes, on='CODIGO_DANE', how='left', suffixes=('', '_csv'))
  # fill Municipio
  merged_df['MUNICIPIO'] = merged_df['MUNICIPIO_csv']
  merged_df = merged_df.drop(columns=['MUNICIPIO_csv'])
  merged_df['CAPITAL'] = merged_df['CODIGO_DANE'].apply(lambda codigo: 1 if codigo[-3:] == '001' else 0)
  # drop sufix (1), (2), (3)
  merged_df['MUNICIPIO'] = merged_df['MUNICIPIO'].str.replace(r'\s*\(\d+\)$', '', regex=True)
  # filter date
  output_df = merged_df[(merged_df['FECHA'] >= '2013-01-01') & (merged_df['FECHA'] <= '2023-12-31')]
  return output_df

In [11]:
filtered_df = clean_data(abigeato_df)

In [12]:
abigeato_output = filtered_df.groupby(["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","CAPITAL"]).sum().reset_index()

In [13]:
abigeato_output.to_csv(OUTPUT_PATH_FILE+"/abigeato.csv",index=False)

## Amenazas

In [14]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [28]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Amenazas']]
amenazas_df = pd.concat(standardized_dataframes, ignore_index=True)

In [29]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','ARMA DE FUEGO','CONTUNDENTES','GRANADA DE MANO',
                             'NO REPORTADO','SIN EMPLEO DE ARMAS','ESCOPOLAMINA']
amenazas_df["AGRUPA_EDAD_PERSONA"] = amenazas_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
amenazas_df["GENERO"] = amenazas_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
amenazas_df["ARMAS_MEDIOS"] = amenazas_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [30]:
amenazas_df['FECHA_TEMP'] = pd.to_datetime(amenazas_df['FECHA'])
amenazas_df['FECHA'] = amenazas_df['FECHA_TEMP'].dt.date
amenazas_df.drop(columns=['FECHA_TEMP'], inplace=True)
amenazas_df['FECHA'] = pd.to_datetime(amenazas_df['FECHA'])

In [31]:
amenazas_filtered_df = clean_data(amenazas_df)

In [32]:
amenazas_output = amenazas_filtered_df.groupby(["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","CAPITAL"]).sum().reset_index()

In [33]:
amenazas_output.to_csv(OUTPUT_PATH_FILE+"/amenazas.csv",index=False)

## Delitos Sexuales

In [34]:
dataframes_by_folder['Delitos_Sexuales'][10]['DEPARTAMENTO_TEMP']=dataframes_by_folder['Delitos_Sexuales'][10]['ARMAS_MEDIOS']
dataframes_by_folder['Delitos_Sexuales'][10]['ARMAS_MEDIOS']=dataframes_by_folder['Delitos_Sexuales'][10]['DEPARTAMENTO']
dataframes_by_folder['Delitos_Sexuales'][10]['DEPARTAMENTO']=dataframes_by_folder['Delitos_Sexuales'][10]['DEPARTAMENTO_TEMP']
dataframes_by_folder['Delitos_Sexuales'][10].drop(columns='DEPARTAMENTO_TEMP',inplace=True)

In [35]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'DELITO', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'DELITO ': 'DELITO',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [36]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Delitos_Sexuales']]
delitos_sexuales_df = pd.concat(standardized_dataframes, ignore_index=True)

In [37]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','CONTUNDENTES','ESCOPOLAMINA','LICOR ADULTERADO','NO REPORTADO',
                             'SIN EMPLEO DE ARMAS','ARMA DE FUEGO','CINTAS/CINTURON','CORTANTES','CORTOPUNZANTES','ESPOSAS','ARMAS BLANCAS']
delitos_sexuales_df["AGRUPA_EDAD_PERSONA"] = delitos_sexuales_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
delitos_sexuales_df["GENERO"] = delitos_sexuales_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
delitos_sexuales_df["ARMAS_MEDIOS"] = delitos_sexuales_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")
delitos_sexuales_df["DELITO"] = delitos_sexuales_df["DELITO"].fillna("NO REPORTADO")

In [38]:
delitos_sexuales_filtered_df = clean_data(delitos_sexuales_df)

In [39]:
delitos_sexuales_output = delitos_sexuales_filtered_df.groupby(["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","DELITO","CAPITAL"]).sum().reset_index()

In [40]:
delitos_sexuales_output.to_csv(OUTPUT_PATH_FILE+"/delitos_sexuales.csv",index=False)